In [1]:
import requests
import os


OUTPUT_DIR = "pruebaa"
os.makedirs(OUTPUT_DIR, exist_ok=True)


datasets = {
    "delitos.csv": "https://servicios.ine.es/wstempus/csv_sc/ES/DATOS_TABLA/26017",
    "capital_formativo.csv": "https://servicios.ine.es/wstempus/csv_sc/ES/DATOS_TABLA/69780",
    "ruido_exterior.csv": "https://servicios.ine.es/wstempus/csv_sc/ES/DATOS_TABLA/74680",
    "gastos_vida.csv": "https://servicios.ine.es/wstempus/csv_sc/ES/DATOS_TABLA/25154",
    
}


for filename, url in datasets.items():    
    response = requests.get(url)
    response.raise_for_status()

    filepath = os.path.join(OUTPUT_DIR, filename)
    with open(filepath, "wb") as f:
        f.write(response.content)


print("Todos los archivos CSV se han descargado correctamente.")


Todos los archivos CSV se han descargado correctamente.


In [ ]:
#asi lo hemos hecho con todos pero para no poner todos hemos puesto esos de ejemplo de como los hemos obtenido.

In [15]:
import pandas as pd
import numpy as np

config = {
    'delitos-csv.csv':        {'peso': 0.18, 'positivo': False, 'col_ccaa': 'Comunidades y Ciudades Autónomas'},
    'desempleo-csv.csv':      {'peso': 0.18, 'positivo': False, 'col_ccaa': 'Comunidades y Ciudades Autónomas'},
    'gastos-vida-csv.csv':    {'peso': 0.18, 'positivo': False, 'col_ccaa': 'Comunidad autónoma de residencia'},
    'salario-medio-csv.csv':  {'peso': 0.12, 'positivo': True,  'col_ccaa': 'Comunidades y Ciudades Autónomas'},
    'demora-listaespera-csv.csv': {'peso': 0.10, 'positivo': False, 'col_ccaa': 'Comunidades y Ciudades Autónomas'},
    'capital-formativo-csv.csv': {'peso': 0.10, 'positivo': True,  'col_ccaa': 'Comunidad autónoma'},
    'accidentes-trafico-csv.csv': {'peso': 0.05, 'positivo': False, 'col_ccaa': 'Comunidades y Ciudades Autónomas'},
    'extranjeros-csv.csv':    {'peso': 0.05, 'positivo': False, 'col_ccaa': 'Comunidades y Ciudades Autónomas'},
    'ruido-exterior-csv.csv': {'peso': 0.04, 'positivo': False, 'col_ccaa': 'Comunidad autónoma'}
}
#lo que hemos hecho es poner un peso segun la importancia de lo que nosotros creemos que son más o menos importantes para vivir en cada comunidad 
#siendo las variables True buenas para la comunidad y False malas para la comunidad.
def limpiar_numero(x):
    if isinstance(x, str):
        x = x.replace('.', '').replace(',', '.')
        x = ''.join(c for c in x if c.isdigit() or c == '.')
    return pd.to_numeric(x, errors='coerce')

def limpiar_ccaa(nombre):
    if not isinstance(nombre, str): return nombre
    nombre = ''.join([i for i in nombre if not i.isdigit()]).strip()
    mapeo = {
        'Castilla - La Mancha': 'Castilla-La Mancha',
        'Asturias, Principado de': 'Asturias',
        'Madrid, Comunidad de': 'Madrid',
        'Murcia, Región de': 'Murcia',
        'Navarra, Comunidad Foral de': 'Navarra',
        'Rioja, La': 'La Rioja',
        'Balears, Illes': 'Islas Baleares',
        'Canarias': 'Canarias'
    }
    return mapeo.get(nombre, nombre)

def normalizar(serie, positivo=True):
    v_min = serie.min()
    v_max = serie.max()
    if positivo:
        return 10 * (serie - v_min) / (v_max - v_min)
    else:
        return 10 * (v_max - serie) / (v_max - v_min)

dfs_limpios = []

for archivo, info in config.items():
    try:
        df = pd.read_csv(archivo, sep=None, engine='python')
        
        if 'capital-formativo' in archivo:
            df = df[df.iloc[:, 0].str.contains('superior', case=False, na=False)]

        df['CCAA'] = df[info['col_ccaa']].apply(limpiar_ccaa)
        df['Valor'] = df['Total'].apply(limpiar_numero)
        
        df = df[~df['CCAA'].str.contains('Nacional', na=False, case=False)]
        
        df_final = df.groupby('CCAA')['Valor'].mean().reset_index()
        
        nombre_col = archivo.replace('-csv.csv', '')
        df_final[nombre_col] = normalizar(df_final['Valor'], positivo=info['positivo'])
        
        dfs_limpios.append(df_final[['CCAA', nombre_col]])
        
    except Exception as e:
        print(f"Error procesando {archivo}: {e}")

ranking_df = dfs_limpios[0]
for next_df in dfs_limpios[1:]:
    ranking_df = pd.merge(ranking_df, next_df, on='CCAA', how='inner')

ranking_df['Score_Final'] = 0
for archivo, info in config.items():
    nombre_col = archivo.replace('-csv.csv', '')
    ranking_df['Score_Final'] += ranking_df[nombre_col] * info['peso']

ranking_final = ranking_df.sort_values(by='Score_Final', ascending=False).reset_index(drop=True)

print("RANKING FINAL DE CCAA:")
print(ranking_final[['CCAA', 'Score_Final']])

#en este ranking estan ceuta y melillaa separados pero para lo final hemos unido ambos.

RANKING FINAL DE CCAA:
                    CCAA  Score_Final
0              Cantabria     7.331336
1                Navarra     7.133315
2             País Vasco     6.890640
3        Castilla y León     6.652338
4               Asturias     6.601822
5               La Rioja     6.532025
6                 Aragón     6.380853
7                Galicia     6.300864
8            Extremadura     6.130788
9     Castilla-La Mancha     6.003514
10               Melilla     5.847230
11              Canarias     5.704652
12                Murcia     5.667803
13        Islas Baleares     5.663502
14                Madrid     5.381862
15                 Ceuta     4.833191
16  Comunitat Valenciana     4.426400
17              Cataluña     4.386872
18             Andalucía     3.296076
